# Cleaning University of Massachussets Amherst Power Consumption Dataset

The solar data retrieved from [UMass Solar Energy](https://www.umass.edu/sustainability/climate-change-energy/solar/15000-solar-panels-5-buildings-2-parking-lots?_ga=2.23708602.1441216145.1605983844-2090397201.1605471456) regarding power consumption in UCB's Mechanical Engineering building is in the form of a `json` format. Here we'll be converting it into a `csv` file for later data analysis. 
The weather data retrieved from [NREL](https://maps.nrel.gov/nsrdb-viewer/)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import pprint
from datetime import datetime

In [3]:
#Format into dates
def format_time(unix_timestamp):
    format = '%Y-%m-%d'
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime(format)

#Create list of datetime objects from timestamps
def gen_datetimes(li):
    datetimes = []
    for time in li:
        datetimes.append(datetime.utcfromtimestamp(time))
    return datetimes

#Get day of week based on date
def get_weekday(dates):
    return [1 if (datetime.strptime(d,"%Y-%m-%d").weekday() >= 5) else 0 for d in dates]

In [6]:
df_weather = pd.read_csv('Datasets/Berkeley_Weather.csv', skiprows=2)
df_weather.head()

,Year,Month,Day,Hour,Minute,DNI,Dew Point,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
0,2019,1,1,0,0,0,-1.2,3.4,0.3,26,74.34,2.9,1000
1,2019,1,1,0,30,0,-1.2,3.4,0.3,26,74.87,2.8,1000
2,2019,1,1,1,0,0,-1.6,3.4,0.3,26,73.13,2.7,1000
3,2019,1,1,1,30,0,-1.6,3.4,0.3,26,73.21,2.7,1001
4,2019,1,1,2,0,0,-2.0,3.5,0.3,27,71.23,2.7,1001


In [7]:
df_weather.describe()

,Year,Month,Day,Hour,Minute,DNI,Dew Point,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
count,17520.0,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000
mean,2019.0,6.526027,15.720548,11.500000,15.000000,260.012614,9.294344,2.474070,1.493647,229.980479,78.368464,13.829697,995.520148
std,0.0,3.447950,8.796498,6.922384,15.000428,359.374979,3.912063,1.157292,0.554094,85.516245,21.518094,5.391254,4.787277
min,2019.0,1.000000,1.000000,0.000000,0.000000,0.000000,-9.900000,0.200000,0.200000,0.000000,11.930000,1.000000,976.000000
25%,2019.0,4.000000,8.000000,5.750000,0.000000,0.000000,7.500000,1.600000,1.100000,191.000000,64.197500,10.100000,993.000000
50%,2019.0,7.000000,16.000000,11.500000,15.000000,0.000000,10.000000,2.300000,1.400000,262.000000,85.680000,12.800000,995.000000
75%,2019.0,10.000000,23.000000,17.250000,30.000000,585.000000,12.000000,3.100000,1.800000,281.000000,96.830000,16.900000,998.000000
max,2019.0,12.000000,31.000000,23.000000,30.000000,1006.000000,17.900000,8.600000,3.500000,360.000000,100.000000,34.700000,1011.000000


In [8]:
df_weather = df_weather.drop(columns=['Hour', 'Minute'])
df_weather = df_weather[df_weather.DNI != 0]
df_weather.head(10)

,Year,Month,Day,DNI,Dew Point,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
15,2019,1,1,45,-2.7,2.9,0.3,33,69.25,2.4,1005
16,2019,1,1,263,-2.3,3.0,0.4,35,66.43,3.4,1005
17,2019,1,1,704,-2.3,3.1,0.4,37,61.93,4.4,1005
18,2019,1,1,802,-2.9,3.2,0.4,39,54.61,5.5,1006
19,2019,1,1,858,-2.9,3.1,0.4,40,50.97,6.5,1006
20,2019,1,1,889,-3.9,2.9,0.5,41,44.07,7.6,1005
21,2019,1,1,916,-3.9,2.6,0.5,38,41.74,8.4,1005
22,2019,1,1,937,-4.8,2.3,0.5,36,36.80,9.2,1005
23,2019,1,1,948,-4.8,2.2,0.5,28,35.11,9.9,1005
24,2019,1,1,952,-5.4,2.1,0.6,21,32.30,10.5,1004


In [9]:
#Take mean, max and min for each DNI in DataFrame and mean for everything else
max_dni = df_weather.groupby(['Year', 'Month', 'Day']).max().reset_index()['DNI']
min_dni = df_weather.groupby(['Year', 'Month', 'Day']).min().reset_index()['DNI']
df_weather = pd.DataFrame(df_weather.groupby(['Year', 'Month', 'Day']).mean().reset_index())
df_weather.insert(4, 'DNI Max', max_dni)
df_weather.insert(5, 'DNI Min', min_dni)
df_weather.head()

,Year,Month,Day,DNI,DNI Max,DNI Min,Dew Point,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
0,2019,1,1,722.400000,952,45,-3.670000,2.120000,0.555000,47.75,43.414000,8.555,1004.55
1,2019,1,2,714.100000,925,67,0.790000,1.465000,1.000000,66.05,57.277500,9.150,1004.40
2,2019,1,3,480.100000,899,14,2.240000,2.040000,0.975000,75.85,64.874500,8.680,1001.75
3,2019,1,4,579.950000,912,30,5.330000,1.675000,0.985000,111.30,73.464000,9.970,994.65
4,2019,1,5,25.388889,93,1,9.966667,6.194444,1.955556,182.50,90.365556,11.500,985.00


In [10]:
df_weather.describe()

,Year,Month,Day,DNI,DNI Max,DNI Min,Dew Point,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure
count,365.0,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,2019.0,6.526027,15.720548,498.180497,780.712329,30.317808,9.059895,2.636308,1.494509,224.030310,66.859900,16.223326,995.686389
std,0.0,3.452584,8.808321,252.846048,270.017108,40.901420,3.986176,1.117535,0.527679,70.040489,16.672748,4.678366,4.606718
min,2019.0,1.000000,1.000000,6.000000,9.000000,1.000000,-7.395455,0.742857,0.286957,21.363636,19.850435,6.516667,979.058824
25%,2019.0,4.000000,8.000000,285.368421,754.000000,3.000000,7.060000,1.734615,1.133333,179.619048,58.423636,12.600000,993.000000
50%,2019.0,7.000000,16.000000,597.142857,919.000000,14.000000,10.057143,2.556250,1.433333,251.000000,69.216250,16.068182,995.333333
75%,2019.0,10.000000,23.000000,714.038462,943.000000,40.000000,11.800000,3.284615,1.862500,270.892857,78.724074,19.406667,998.192308
max,2019.0,12.000000,31.000000,819.478261,1006.000000,232.000000,16.160714,6.340909,3.232000,349.238095,96.725000,29.032000,1009.550000


In [12]:
df_weather.to_csv('Amherst_Weather_Cleaned')